In [1]:
import carla

In [2]:
# Create a client that will connect to the CARLA server running at localhost on port 2000
client = carla.Client('localhost', 2000)

# Get the world object (the simulation environment) from the client
world = client.get_world()

# Get all possible spawn points in the world
spawn_points = world.get_map().get_spawn_points()

# Get the blueprint for a firetruck from the world's blueprint library
vehicle_bp = world.get_blueprint_library().filter("firetruck*")

# Select the first spawn point from the list of spawn points
start_point = spawn_points[0]

# Try to spawn a firetruck at the first spawn point
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

In [3]:
print(type(vehicle))

<class 'carla.libcarla.Vehicle'>


In [4]:
# get the car's position on the map
vehicle_pos = vehicle.get_transform()
print(vehicle_pos)

Transform(Location(x=-64.644844, y=24.471010, z=0.580205), Rotation(pitch=0.000000, yaw=0.159198, roll=0.000000))


In [5]:
# send the vehicle off
vehicle.set_autopilot(True)

In [6]:
# get the actual position from the car moving
vehicle_pos = vehicle.get_transform()
print(vehicle_pos)

Transform(Location(x=-63.826649, y=24.397814, z=-0.003055), Rotation(pitch=0.148133, yaw=-1.305939, roll=0.113880))


In [7]:
# now look at the map
town_map = world.get_map()

In [8]:
type(town_map)

carla.libcarla.Map

In CARLA, the `get_topology()` method of a `Map` object returns a list of pairs of `Waypoint` objects. Each pair represents a road segment, where the first `Waypoint` is the beginning of the road segment and the second `Waypoint` is the end of the road segment.

So, if `roads = town_map.get_topology()`, then `roads` is a list of tuples, where each tuple contains two `Waypoint` objects representing a road segment in the map.

Here's an example of what a single item in `roads` might look like:

```python

(Waypoint(x=2.5, y=3.5, z=0.0, pitch=0.0, yaw=180.0, roll=0.0, id=1, road_id=1, section_id=1, lane_id=1, s=0.0, transform=Transform(Location(x=2.5, y=3.5, z=0.0), Rotation(pitch=0.0, yaw=180.0, roll=0.0)), lane_type=LaneType.Driving, lane_change=LaneChange.Both, lane_width=3.5, right_lane_marking=LaneMarking(type=White, color=White, lane_change=Both), left_lane_marking=LaneMarking(type=White, color=White, lane_change=Both)),

 Waypoint(x=2.5, y=4.5, z=0.0, pitch=0.0, yaw=180.0, roll=0.0, id=2, road_id=1, section_id=1, lane_id=1, s=1.0, transform=Transform(Location(x=2.5, y=4.5, z=0.0), Rotation(pitch=0.0, yaw=180.0, roll=0.0)), lane_type=LaneType.Driving, lane_change=LaneChange.Both, lane_width=3.5, right_lane_marking=LaneMarking(type=White, color=White, lane_change=Both), left_lane_marking=LaneMarking(type=White, color=White, lane_change=Both)))
 ```
 


In this example, the first `Waypoint` represents a point on the road at `(x=2.5, y=3.5, z=0.0)`, and the second `Waypoint` represents a point on the same road one meter ahead at `(x=2.5, y=4.5, z=0.0)`.

In [9]:
roads = town_map.get_topology()
print(roads[0][0])

Waypoint(Transform(Location(x=109.929878, y=-9.334196, z=0.000000), Rotation(pitch=0.000000, yaw=-89.609253, roll=0.000000)))


In [10]:
len(roads) # -> there are 200 road segments in the current town map of the simulation

200

In [25]:
# import some code coming with the simulator
import sys
import time
sys.path.append('C:/CARLA/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [26]:
# using the code to plan a route and then draw it in the simulator
town10hd_map = world.get_map()

sampling_resolution = 2

grp = GlobalRoutePlanner(town_map, sampling_resolution)

# define the two ending points of the route
point_a = carla.Location(x = -64.644844, y = 24.471010, z = 0.600000)
point_b = carla.Location(x = -113.903656, y = 14.422489, z = -0.003719)

route = grp.trace_route(point_a, point_b) # create the route to map from point a to point b

for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow = False, color = carla.Color(r = 0, g = 0, b = 255), life_time = 230.0, persistent_lines = True)

In [27]:
# utility script of destruction

for actor in world.get_actors().filter("*vehicle*"):
    actor.destroy()
for sensor in world.get_actors().filter("*sensor*"):
    sensor.destroy()

In [30]:
# now we define 2 cars
truck_bp = world.get_blueprint_library().filter("*firetruck*")
mini_bp = world.get_blueprint_library().filter("*cooper_s*")

# start the first car in already define start point
truck = world.try_spawn_actor(truck_bp[0], start_point)

In [31]:
# tweak the spectator position to watch the show

spectator = world.get_spectator() # get the spectator object
spawn_points = world.get_map().get_spawn_points() # get all spawn points
start_point = spawn_points[0] # select the first spawn point

spectator_pos = carla.Transform(start_point.location + carla.Location(x=20,y=10,z=4),
                                carla.Rotation(yaw = start_point.rotation.yaw -155)) # define the new spectator position

spectator.set_transform(spectator_pos)  # move the spectator to the new position

In [32]:
# drop the minicooper from the sky to see what happends

# spawn it from somewhere
mini = world.try_spawn_actor(mini_bp[0], spawn_points[10]) # spawn the mini cooper at spawn point 10

mini_pos = carla.Transform(start_point.location + carla.Location(x=-4,z=10),
                            carla.Rotation(yaw = start_point.rotation.yaw - 0)) # define the new spectator position

mini.set_transform(mini_pos)    # move the mini cooper to the new position

In [35]:
# utility script of destruction

for actor in world.get_actors().filter("*vehicle*"):
    actor.destroy()
for sensor in world.get_actors().filter("*sensor*"):
    sensor.destroy()

In [36]:
import time

# redefine 2 cars
truck_bp = world.get_blueprint_library().filter("*firetruck*")
mini_bp = world.get_blueprint_library().filter("*cooper_s*")

# start the first car in already define start point
truck = world.try_spawn_actor(truck_bp[0], start_point)
time.sleep(3) # wait 3 seconds

vw_bp = world.get_blueprint_library().filter("*volkswagen*")

vw = world.try_spawn_actor(vw_bp[0], spawn_points[20]) # spawn the volkswagen at spawn point 20
vw_pos = carla.Transform(start_point.location + carla.Location(x=1,z=6),
                            carla.Rotation(yaw = start_point.rotation.yaw - 0)) # define the new spectator position
vw.set_transform(vw_pos)    # move the volkswagen to the new position


mini = world.try_spawn_actor(mini_bp[0], spawn_points[10]) # spawn the mini cooper at spawn point 10
mini_pos = carla.Transform(start_point.location + carla.Location(x=1,z=10),
                            carla.Rotation(yaw = start_point.rotation.yaw - 0)) # define the new spectator position
mini.set_transform(mini_pos)   # move the mini cooper to the new position